In [ ]:
import requests
from bs4 import BeautifulSoup
from astropy.io import fits
import matplotlib.pyplot as plt
from desispec.io import read_spectra
from desispec.coaddition import coadd_cameras
import numpy as np
from desitarget.targetmask import desi_mask
from astropy.table import Table
from desispec.resolution import Resolution
from scipy.optimize import curve_fit
import pandas as pd


In [ ]:
hdul = fits.open('QSO_cat_fuji_healpix_only_qso_targets.fits')
data = hdul[1].data

df = pd.read_csv('MgII-ParentQSO-EDR.csv')

In [ ]:
parent_cat_set = set(df['TARGETID'])

tid_array = []
healpix_array = []
for i, tid in enumerate(data['TARGETID']):
    if tid in parent_cat_set and data['PROGRAM'][i] == 'dark': 
        tid_array.append(tid)
        healpix_array.append(data['HPXPIXEL'][i])

    
print(len(tid_array), len(healpix_array))
healpix_set = set(healpix_array)

In [ ]:
all_links = []
survey_array = []
all_healpix = []
for i in ["sv1", "sv2", "sv3"]:
    r1 = requests.get(f'https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/{i}/dark/')
    print(i)
    soup = BeautifulSoup(r1.content, 'html.parser')
    links = soup.find_all('a')
    linkList = [link['href'] for link in links[1:]]
    for ii in range(len(linkList)):
        r2 = requests.get(f'https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/{i}/dark/{linkList[ii]}')
        # print(r2.status_code)
        soup = BeautifulSoup(r2.content, 'html.parser')
        links2 = soup.find_all('a')
        linkList2 = [link['href'] for link in links2[1:]]
        for link in linkList2:
            link = link.replace('/','')
            if int(link) in healpix_set:
                # print(link)
                all_links.append(f'https://data.desi.lbl.gov/public/edr/spectro/redux/fuji/healpix/{i}/dark/{linkList[ii]}{link}')
                all_healpix.append(link)
                survey_array.append(i)


In [ ]:
healpix_check = set()
zArray = []
RaArray = []
DecArray = []
tidArray = []
def gauss(x,sig,A,mu): # gaussian function
    return A*np.exp(-((x-mu)**2) / (2*sig**2))

In [ ]:
for i in range(1):
    if all_healpix[i] in healpix_check:
        print('skip')
        continue
    else:
        downfile = f'{all_links[i]}/coadd-{survey_array[i]}-dark-{all_healpix[i]}.fits'
        print(downfile)
        !wget "{downfile}" 
        print(all_healpix[i])

        spec = read_spectra(f'coadd-{survey_array[i]}-dark-{all_healpix[i]}.fits')
        fibermap = spec.fibermap
        coadd_dict = {tid: ii for ii, tid in enumerate(fibermap["TARGETID"])} # creates a dictionary of indicies with the target id as the key
        for j, tid in enumerate(data["TARGETID"]):
            if tid in coadd_dict and tid in parent_cat_set:
                tidArray.append(str(tid)) #generates a list of target id's of valid QSOs -> saved as str because of floating point problems
                zArray.append(data["Z"][j]) # generates a list of redshift of all the valid QSOs
                RaArray.append(data["TARGET_RA"][j]) # generates a list of RA of all the valid QSOs
                DecArray.append(data["TARGET_DEC"][j]) # generates a list of DEC of all the valid QSOs
                spec_one = spec[coadd_dict[tid]]
                wave = coadd_cameras(spec_one).wave['brz'] # extracts and combines the three arms for wave, flux, and ivar     
                flux = coadd_cameras(spec_one).flux['brz'][0]
                ivar = coadd_cameras(spec_one).ivar['brz'][0]
                pixel_mask = coadd_cameras(spec_one).mask['brz'][0] # saves array of masked/unmasked pixels

                res = coadd_cameras(spec_one).resolution_data['brz'][0] # gets resolution data from current spectra
                R  = Resolution(res) # creates sparse resolution matrix
                sigArray = []
                rArray = []
                for jj in range(len(wave)): # generates
                    kernel = res[:, jj]  # shape: (ndiag,), centered at wavejj]
                    # Compute the pixel offsets relative to center
                    offsets = R.offsets
                    kernel_wave = wave[jj] + (offsets * np.gradient(wave)[jj])
                    if np.nan in kernel:
                        sigArray.append(np.nan)
                    elif np.inf in kernel:
                        sigArray.append(np.inf)
                    else:
                        # Get the resolution kernel (ISF) at pixel jj
                        kernel = res[:, jj]  # shape: (ndiag,), centered at wave[jj]
                        # Compute the pixel offsets relative to center
                        offsets = R.offsets
                        kernel_wave = wave[jj] + (offsets * np.gradient(wave)[jj])
                        peak = max(kernel)
                        parameters = curve_fit(gauss, kernel_wave, kernel, p0=[0.5, 1, kernel_wave[5]])
                        sig_fit, A_fit, mu_fit = parameters[0]
                        sigArray.append(sig_fit)
                t = Table()
                t['WAVE'] = wave # formats the data into a table
                t['FLUX'] = flux                        
                t['IVAR'] = ivar
                t['PIXEL_MASK'] = pixel_mask 
                t['SIGMA_PIXEL'] = sigArray
                t.write(str(tid) + '.fits',"overwrite=True") # writes the data to a fits file for analysis
        healpix_check.add(all_healpix[i])
        filename = f'coadd-{survey_array[i]}-dark-{all_healpix[i]}.fits'
        !rm -rf "{filename}" # removes the downloaded coadd file
t2 = Table()
t2['TARGETID'] = tidArray 
t2['Z'] = zArray
t2['TARGET_RA'] = RaArray
t2['TARGET_DEC'] = DecArray
t2.write('QSO_mgii_parent_catalog.fits',"overwrite=True") # writes a file with tid and redshifts of all the valid QSOs



    

In [ ]:
f'coadd-{survey_array[i]}-dark-{all_healpix[i]}.fits'